In [2]:
from azureml.core import Workspace
from azureml.core.webservice import AciWebservice
from azureml.core.webservice import Webservice
from azureml.core.model import InferenceConfig
from azureml.core.environment import Environment
from azureml.core import Workspace
from azureml.core.model import Model
from azureml.core.conda_dependencies import CondaDependencies

In [3]:
ws = Workspace.from_config()

In [4]:
model = Model.register(workspace = ws,
              model_path ="outputs/model.pkl",
              model_name = "base",
              tags = {"version": "1"},
              description = "logistic task",
              )

Registering model base


In [5]:
env = Environment('envnew')
cd = CondaDependencies.create(pip_packages=['pandas==1.1.5', 'azureml-defaults','joblib==0.17.0'], conda_packages = ['scikit-learn==0.23.2'])
env.python.conda_dependencies = cd
# Register environment to re-use later
env.register(workspace = ws)
print("Registered Environment")

Registered Environment


In [6]:
myenv = Environment.get(workspace=ws, name="envnew")

In [7]:
myenv.save_to_directory('./environ', overwrite=True)

In [8]:
aciconfig = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=1)

In [9]:
inference_config = InferenceConfig(entry_script="score.py", environment=myenv)

In [12]:
service = Model.deploy(workspace=ws,
                name='baseservice',
                models=[model],
                inference_config=inference_config,
                deployment_config=aciconfig, 
                overwrite = True)
service.wait_for_deployment(show_output=True)
url = service.scoring_uri
print(url)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2021-07-16 16:50:34+00:00 Creating Container Registry if not exists.
2021-07-16 16:50:34+00:00 Registering the environment.
2021-07-16 16:50:35+00:00 Use the existing image.
2021-07-16 16:50:35+00:00 Generating deployment configuration.
2021-07-16 16:50:36+00:00 Submitting deployment to compute.
2021-07-16 16:50:38+00:00 Checking the status of deployment baseservice.